### K-means clustering

In [2]:
import numpy as np

def parseVector(line):
    return np.array([float(x) for x in line.split()])

def closestPoint(p, centers):
    bestIndex = 0
    closest = float("+inf")
    for i in range(len(centers)):
        tempDist = np.sum((p - centers[i]) ** 2)
        if tempDist < closest:
            closest = tempDist
            bestIndex = i
    return bestIndex

lines = sc.textFile('data/kmeans_bigdata.txt', 5)  

# The data file can be downloaded at http://www.cse.ust.hk/msbd5003/data/kmeans_bigdata.txt
# lines = sc.textFile('../data/kmeans_bigdata.txt', 5)  
# lines is an RDD of strings
K = 3
convergeDist = 0.01  
# terminate algorithm when the total distance from old center to new centers is less than this value

data = lines.map(parseVector).cache() # data is an RDD of arrays

kCenters = data.takeSample(False, K, 1)  # intial centers as a list of arrays
tempDist = 1.0  # total distance from old centers to new centers

while tempDist > convergeDist:
    closest = data.map(lambda p: (closestPoint(p, kCenters), (p, 1)))
    # for each point in data, find its closest center
    # closest is an RDD of tuples (index of closest center, (point, 1))
        
    pointStats = closest.reduceByKey(lambda p1, p2: (p1[0] + p2[0], p1[1] + p2[1]))
    # pointStats is an RDD of tuples (index of center,
    # (array of sums of coordinates, total number of points assigned))
    
    newCenters = pointStats.map(lambda st: (st[0], st[1][0] / st[1][1])).collect()
    # compute the new centers
    
    tempDist = sum(np.sum((kCenters[i] - p) ** 2) for (i, p) in newCenters)
    # compute the total disctance from old centers to new centers
    
    for (i, p) in newCenters:
        kCenters[i] = p
        
print("Final centers: ", kCenters)


Final centers:  [array([0.82604552, 0.5360665 ]), array([0.3917835 , 0.17797951]), array([0.31331048, 0.70637383])]


### PageRank

In [3]:
import re
from operator import add

def computeContribs(urls, rank):
    # Calculates URL contributions to the rank of other URLs.
    num_urls = len(urls)
    for url in urls:
        yield (url, rank / num_urls)

def parseNeighbors(urls):
    # Parses a urls pair string into urls pair."""
    parts = urls.split(' ')
    return parts[0], parts[1]

# Loads in input file. It should be in format of:
#     URL         neighbor URL
#     URL         neighbor URL
#     URL         neighbor URL
#     ...

lines = sc.textFile("data/pagerank_data.txt", 2)
# lines = sc.textFile("../data/dblp.in", 5)

numOfIterations = 10

# Loads all URLs from input file and initialize their neighbors. ''
links = lines.map(lambda urls: parseNeighbors(urls)) \
             .groupByKey()

# Loads all URLs with other URL(s) link to from input file 
# and initialize ranks of them to one.
ranks = links.mapValues(lambda neighbors: 1.0)

# Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(numOfIterations):
    # Calculates URL contributions to the rank of other URLs.
    contribs = links.join(ranks) \
                    .flatMap(lambda url_urls_rank:
                             computeContribs(url_urls_rank[1][0],
                                             url_urls_rank[1][1]))
    # After the join, each element in the RDD is of the form
    # (url, (list of neighbor urls, rank))
    print("links:", links.join(ranks).collect())
    # Re-calculates URL ranks based on neighbor contributions.
    ranks = contribs.reduceByKey(add).mapValues(lambda rank: rank * 0.85 + 0.15)
    # ranks = contribs.reduceByKey(add).map(lambda t: (t[0], t[1] * 0.85 + 0.15))
    print("rank:", ranks.collect())
print(ranks.top(5, lambda x: x[1]))

links: [('1', (<pyspark.resultiterable.ResultIterable object at 0x7f552a032520>, 1.0)), ('4', (<pyspark.resultiterable.ResultIterable object at 0x7f5529f3b160>, 1.0)), ('2', (<pyspark.resultiterable.ResultIterable object at 0x7f5529f3b190>, 1.0)), ('3', (<pyspark.resultiterable.ResultIterable object at 0x7f5529f3b1c0>, 1.0))]
rank: [('1', 1.4249999999999998), ('4', 1.0), ('2', 0.575), ('3', 1.0)]
links: [('1', (<pyspark.resultiterable.ResultIterable object at 0x7f552a032670>, 1.4249999999999998)), ('4', (<pyspark.resultiterable.ResultIterable object at 0x7f5529ec1790>, 1.0)), ('2', (<pyspark.resultiterable.ResultIterable object at 0x7f5529ec1940>, 0.575)), ('3', (<pyspark.resultiterable.ResultIterable object at 0x7f5529ec15b0>, 1.0))]
rank: [('1', 1.244375), ('4', 1.0), ('2', 0.7556249999999999), ('3', 0.9999999999999999)]
links: [('1', (<pyspark.resultiterable.ResultIterable object at 0x7f5529f2d8e0>, 1.244375)), ('4', (<pyspark.resultiterable.ResultIterable object at 0x7f5529ec7520>,

### PMI

In [4]:
lines = sc.textFile('data/adj_noun_pairs.txt', 8)

In [5]:
lines.count()

3162692

In [6]:
lines.take(5)

['early radical',
 'french revolution',
 'pejorative way',
 'violent means',
 'positive label']

In [7]:
# Converting lines into word pairs. 
# Data is dirty: some lines have more than 2 words, so filter them out.
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

PythonRDD[161] at RDD at PythonRDD.scala:53

In [8]:
N = pairs.count()
print(N)

3162674


In [9]:
# Compute the frequency of each pair.
# Ignore pairs that not frequent enough
pair_freqs = pairs.map(lambda p: (p,1)).reduceByKey(lambda f1, f2: f1 + f2)\
                  .filter(lambda pf: pf[1] >= 100)
pair_freqs.take(5)

[(('16th', 'century'), 950),
 (('civil', 'war'), 2236),
 (('social', 'class'), 155),
 (('sixteenth', 'century'), 137),
 (('late', '1970'), 444)]

In [10]:
# Computing the frequencies of the adjectives and the nouns
a_freqs = pairs.map(lambda p: (p[0],1)).reduceByKey(lambda x,y: x+y)
n_freqs = pairs.map(lambda p: (p[1],1)).reduceByKey(lambda x,y: x+y)

n_freqs.take(5)

[('revolution', 2305),
 ('wealth', 363),
 ('idea', 2852),
 ('anarchism', 67),
 ('something', 744)]

In [11]:
print(n_freqs.count())
print(a_freqs.count())
print(pair_freqs.count())

106333


104304
2012


In [12]:
# Broadcasting the adjective and noun frequencies. 
#a_dict = a_freqs.collectAsMap()
#a_dict = sc.parallelize(a_dict).map(lambda x: x)
#a_dict.value['violent']
n_dict = n_freqs.collectAsMap()
a_dict = a_freqs.collectAsMap()
a_dict['violent']

1191

In [13]:
from math import *

# Computing the PMI for a pair.
def pmi_score(pair_freq):
    w1, w2 = pair_freq[0]
    f = pair_freq[1]
    pmi = log(float(f)*N/(a_dict[w1]*n_dict[w2]), 2)
    return pmi, (w1, w2)

# Computing the PMI for all pairs.
scored_pairs = pair_freqs.map(pmi_score)

# Printing the most strongly associated pairs. 
scored_pairs.top(10)

[(14.41018838546462, ('magna', 'carta')),
 (13.071365888694997, ('polish-lithuanian', 'Commonwealth')),
 (12.990597616733414, ('nitrous', 'oxide')),
 (12.64972604311254, ('latter-day', 'Saints')),
 (12.50658937509916, ('stainless', 'steel')),
 (12.482331020687814, ('pave', 'runway')),
 (12.19140721768055, ('corporal', 'punishment')),
 (12.183248694293388, ('capital', 'punishment')),
 (12.147015483562537, ('rush', 'yard')),
 (12.109945794428935, ('globular', 'cluster'))]

# Divide-and-Conqueror

### Prefix Sums

In [14]:
x = [1, 4, 3, 5, 6, 7, 0, 1]

rdd = sc.parallelize(x, 4).cache()

print(rdd.glom().collect())

def f(iterator):
    yield sum(iterator)

sums = rdd.mapPartitions(f).collect()

print(sums)

for i in range(1, len(sums)):
    sums[i] += sums[i-1]

print(sums)

def g(index, iterator):
    global sums
    if index == 0:
        s = 0
    else:
        s = sums[index-1]
    for i in iterator:
        s += i
        yield s

prefix_sums = rdd.mapPartitionsWithIndex(g)
print(prefix_sums.collect())

[[1, 4], [3, 5], [6, 7], [0, 1]]
[5, 8, 13, 1]
[5, 13, 26, 27]
[1, 5, 8, 13, 19, 26, 26, 27]


### Monotonicity checking

In [15]:
x = [1, 3, 4, 5, 7, 3, 10, 14, 16, 20, 21, 24, 24, 26, 27, 30]

rdd = sc.parallelize(x, 4).cache()
print(rdd.glom().collect())

def f(it):
    first = next(it)
    last = first
    increasing = True
    for i in it:
        if i < last:
            increasing = False
        last = i
    yield increasing, first, last

results = rdd.mapPartitions(f).collect()

print(results)

increasing = True
if results[0][0] == False:
    increasing = False
else:
    for i in range(1, len(results)):
        if results[i][0] == False or results[i][1] < results[i-1][2]:
            increasing = False
if increasing:
    print("Monotone")
else:
    print("Not monotone")


[[1, 3, 4, 5], [7, 3, 10, 14], [16, 20, 21, 24], [24, 26, 27, 30]]
[(True, 1, 5), (False, 7, 14), (True, 16, 24), (True, 24, 30)]
Not monotone


### 2D Maxima
The sequential algorithm: https://en.wikipedia.org/wiki/Maxima_of_a_point_set 

In [16]:
numPartitions = 10

points = sc.textFile('data/points.txt',numPartitions)
pairs = points.map(lambda l: tuple(l.split()))
pairs = pairs.map(lambda pair: (int(pair[0]),int(pair[1]))).sortByKey(False)

def findmaxY(l):
    m = 0
    for x in l:
        if x[1] > m:
            m = x[1]
    yield m

pairs.cache()
maxY = pairs.mapPartitions(findmaxY).collect()
for i in range(1, len(maxY)):
    if maxY[i-1] > maxY[i]:
        maxY[i] = maxY[i-1]

def findmaxima(i, l):
    if i == 0:
        m = 0
    else:
        m = maxY[i-1]
    for x in l:
        if x[1] > m:
            m = x[1]
            yield x

maxima = pairs.mapPartitionsWithIndex(findmaxima)
            
print(maxima.collect())


[(99990, 53983), (99977, 59457), (99958, 59581), (99956, 79605), (99885, 83645), (99827, 89705), (99740, 98217), (99662, 98973), (96815, 99398), (96167, 99795), (92693, 99960), (92580, 99974), (59920, 99992), (45834, 100000)]


### Maximum Subarray Problem

In [17]:
# Classical divide and conquer algorithm

A = [-3, 2, 1, -4, 5, 2, -1, 3, -1]

def MaxSubarray(A, p, r):
    if p == r:
        return A[p]
    q = (p+r)//2
    M1 = MaxSubarray(A, p, q)
    M2 = MaxSubarray(A, q+1, r)
    Lm = -float('inf')
    Rm = Lm
    V = 0
    for i in range(q, p-1, -1):
        V += A[i]
        if V > Lm:
            Lm = V
    V = 0
    for i in range(q+1, r+1):
        V += A[i]
        if V > Rm:
            Rm = V
    return max(M1, M2, Lm+Rm)

print(MaxSubarray(A, 0, len(A)-1))

9


In [18]:
# Linear-time algorithm
# Written in a way so that we can call it for each partition

def linear_time(it):
    Vmax = -float('inf')
    V = 0
    for Ai in it:
        V += Ai
        if V < Ai:
            V = Ai
        if V > Vmax:
            Vmax = V
    yield Vmax
    
print(next(linear_time(A)))

9


In [19]:
# The Spark algorithm:

def compute_sum(it):
    yield sum(it)

def compute_LmRm(index, it):
    Rm = -float('inf')
    L = sums[index]
    Lm = L
    R = 0
    for Ai in it:
        L -= Ai
        R += Ai
        if L > Lm:
            Lm = L
        if R > Rm:
            Rm = R
    yield (Lm, Rm)

num_partitions = 4
rdd = sc.parallelize(A, num_partitions).cache()
sums = rdd.mapPartitions(compute_sum).collect()
print(sums)
LmRms = rdd.mapPartitionsWithIndex(compute_LmRm).collect()
print(LmRms)
best = max(rdd.mapPartitions(linear_time).collect())

for i in range(num_partitions-1):
    for j in range(i+1, num_partitions):
        x = LmRms[i][0] + sum(sums[i+1:j]) + LmRms[j][1]
        if x > best:
            best = x

print(best)

[-1, -3, 7, 1]
[(2, -1), (0, 1), (7, 7), (2, 2)]
9


### Sample Sort

In [20]:
import random

n = 100 # size of RDD to be sorted
p = 8   # number of partitions/workers
s = 3  # sample size factor 

r = sc.parallelize([random.randrange(1, n*100, 1) for i in range(n)], p)

sample = r.takeSample(False, s*p)
sample.sort()
splitters = [0] + sample[s::s] 
print(splitters)

def findBucket(par):
    buckets = [[] for i in range(p)]
    for x in par:
        # Do a binary search
        l, r = 0, p-1
        while l < r:
            mid = (l+r+1)//2
            if splitters[mid] < x:
                l = mid
            else:
                r = mid-1
        buckets[l].append(x)
    for i in range(p):
        yield (i, buckets[i])

def sortBucket(b):
    l = []
    for x in b[1]:
        l += x
    l.sort()
    return l
        
r1 = r.mapPartitions(findBucket)
r2 = r1.groupByKey()
r3 = r2.flatMap(sortBucket)

print(r.glom().collect())
print(r1.glom().collect())
print(r3.glom().collect())

[0, 2046, 2561, 3206, 3720, 5499, 7527, 8887]
[[2607, 7280, 3984, 7984, 2179, 5065, 6471, 5499, 5501, 3067, 9834, 9147], [605, 2641, 7432, 516, 4491, 3240, 1716, 5578, 2617, 9481, 4989, 6655], [941, 7795, 4256, 2138, 674, 2175, 2919, 6353, 3480, 5562, 8033, 9292], [8887, 9460, 3100, 854, 8748, 4515, 8579, 9374, 4619, 8693, 5079, 3836], [1875, 5427, 3206, 6999, 2561, 9359, 7241, 8352, 2710, 8036, 167, 2046], [4484, 1062, 9319, 5382, 2406, 559, 4488, 6605, 8460, 542, 9387, 4394], [8874, 8660, 974, 3335, 2256, 2519, 3655, 1208, 2208, 3275, 5284, 3370], [5247, 4730, 1527, 2621, 2300, 3720, 5336, 6032, 8763, 9500, 7527, 4558, 7101, 5720, 6061, 9598]]
[[(0, []), (1, [2179]), (2, [2607, 3067]), (3, []), (4, [3984, 5065, 5499]), (5, [7280, 6471, 5501]), (6, [7984]), (7, [9834, 9147])], [(0, [605, 516, 1716]), (1, []), (2, [2641, 2617]), (3, [3240]), (4, [4491, 4989]), (5, [7432, 5578, 6655]), (6, []), (7, [9481])], [(0, [941, 674]), (1, [2138, 2175]), (2, [2919]), (3, [3480]), (4, [4256]), (5,

### String lexicographic Comparison

In [21]:
x = 'abcccbcbcacaccacaabb'
y = 'abcccbcccacaccacaabb'

numPartitions = 4
rdd = sc.parallelize(zip(x,y), numPartitions)

def compareStrings(string_pairs):
    for x, y in string_pairs:
        if x < y:
            return '<'
        elif x > y:
            return '>'
    return '='

comparisons = rdd.mapPartitions(compareStrings).filter(lambda sign: sign != '=')
if not comparisons.isEmpty():
    result = comparisons.first()
else:
    result = "="

print(result)

<


### Find the first (adj, noun) pair in which the noun is 'unification'.

In [22]:
numPartitions = 10
path_to_file = "data/adj_noun_pairs.txt"

lines = sc.textFile(path_to_file, numPartitions)
pairs = lines.map(lambda l: tuple(l.split())).filter(lambda p: len(p)==2)
pairs.cache()

def returnFirst(pairs):
    txt = 'unification'
    for pair in pairs:
        adj, noun = pair
        if noun == txt:
            return [adj]
    return ""

first_adj = pairs.mapPartitions(returnFirst).filter(lambda x: x != "").first()
print(first_adj)

24/05/19 18:27:06 WARN BlockManager: Task 382 already completed, not releasing lock for rdd_222_1
24/05/19 18:27:06 WARN BlockManager: Task 384 already completed, not releasing lock for rdd_222_3


24/05/19 18:27:07 WARN BlockManager: Task 383 already completed, not releasing lock for rdd_222_2
24/05/19 18:27:07 WARN BlockManager: Task 385 already completed, not releasing lock for rdd_222_4
several


In [ ]:
''